In [1]:
import numpy as np
import cvxpy as cp
from qsys import sigma_0, sigma_x, sigma_y, sigma_z
import qcvx

I, X, Y, Z = sigma_0, sigma_x, sigma_y, sigma_z

'''
The unitary gates might not be normalized
'''
S0 = [
    (I, I), (I, X), (I, Z),
    (X, I), (X, X), (X, Z),
    (Z, I), (Z, X), (Z, Z),
    (X+Y, X-Y), (X-Y, X+Y),
    (Z+Y, Z-Y), (Z-Y, Z+Y),
    # (Y, Y),
    # (I+1j*Y, I+1j*Y), (I-1j*Y, I-1j*Y),
]

S1 = [
    (Y, I), (Y, X), (Y, Z),
    (I, Y), (X, Y), (Z, Y),
    (I+1j*Y, I-1j*Y), (I-1j*Y, I+1j*Y),
    # (X+Y, X+Y), (X-Y, X-Y),
    # (Z+Y, Z+Y), (Z-Y, Z-Y),
]

def choi_gates(p):
    # Choi(p[0]) tensor Choi(p[1])
    v = np.einsum(p[0], range(2), p[1], range(2,4))
    x = np.einsum(v, range(4), v.conjugate(), range(4,8)).reshape(2**4,-1)
    return x * 2**2 / np.trace(x)

def solve_sdp(worst_prob, avg_prob, strategy, precision = 1e-6):
    print(f'{strategy} ({precision})')
    worst_prob.solve(solver=cp.SCS, eps=precision)
    if worst_prob.status == 'optimal':
        print('    worst-case:', worst_prob.value)
    else:
        print(f'Failed to get the worst-case optimal (status: {worst_prob.status}).')
    avg_prob.solve(solver=cp.SCS, eps=precision)
    if avg_prob.status == 'optimal':
        print('    average:', avg_prob.value)
    else:
        print(f'Failed to get the average optimal (status: {avg_prob.status})')

Numbering the systems
- gate U from system 0 to system 1
- gate V from system 2 to system 3

In [2]:
def causal_strategy():
    d = 2
    dims = [d] * 4
    D = np.prod(dims)
    tr_supermap = d**2

    S = cp.Variable((D, D), hermitian=True)
    # {P, S-P} is a tester
    P = cp.Variable((D, D), hermitian=True)
    p_err = cp.Variable()

    def ptrace(S, trace_list):
        return qcvx.partial_trace(S, dims, trace_list, discard=False)

    cons = [
        P >> 0,
        S >> P,
        cp.trace(S) == tr_supermap,
        S == ptrace(S, [3]),
        ptrace(S, [2,3]) == ptrace(S, [1,2,3]),
    ]

    # probability of the measurement outcome
    def pr(x):
        return cp.real(cp.trace(P @ choi_gates(x)))

    # the worst-case error probability
    worst_cons = [pr(x) >= 1-p_err for x in S0]
    worst_cons += [pr(x) <= p_err for x in S1]

    # the avarage error probability
    avg_cons = [
        p_err == sum([1-pr(x) for x in S0] + [pr(x) for x in S1]) / (len(S0) + len(S1))
    ]

    return cp.Problem(cp.Minimize(p_err), cons + worst_cons), cp.Problem(cp.Minimize(p_err), cons + avg_cons)

solve_sdp(*causal_strategy(), 'causal strategy')

causal strategy (1e-06)
    worst-case: 0.12368812202896791
    average: 0.09442961439480095


In [3]:
def non_causal_strategy():
    d = 2
    dims = [d] * 4
    D = np.prod(dims)
    tr_supermap = d**2

    S = cp.Variable((D, D), hermitian=True)
    # {P, S-P} is a tester
    P = cp.Variable((D, D), hermitian=True)
    p_err = cp.Variable()

    def ptrace(S, trace_list):
        return qcvx.partial_trace(S, dims, trace_list, discard=False)

    cons = [
        P >> 0,
        S >> P,
        cp.trace(S) == tr_supermap,
        S == ptrace(S, [3]) + ptrace(S, [1]) - ptrace(S, [1,3]),
        ptrace(S, [0,1]) == ptrace(S, [0,1,3]),
        ptrace(S, [2,3]) == ptrace(S, [1,2,3]),
    ]

    # probability of the measurement outcome
    def pr(x):
        return cp.real(cp.trace(P @ choi_gates(x)))

    # the worst-case error probability
    worst_cons = [pr(x) >= 1-p_err for x in S0]
    worst_cons += [pr(x) <= p_err for x in S1]

    # the avarage error probability
    avg_cons = [
        p_err == sum([1-pr(x) for x in S0] + [pr(x) for x in S1]) / (len(S0) + len(S1))
    ]

    return cp.Problem(cp.Minimize(p_err), cons + worst_cons), cp.Problem(cp.Minimize(p_err), cons + avg_cons)

solve_sdp(*non_causal_strategy(), 'strategy with indefinite causal order')

strategy with indefinite causal order (1e-06)
    worst-case: 0.11214883458520188
    average: 0.08025490396026533


In [4]:
def non_causal_strategy2():
    d = 4
    dims = [d] * 4
    D = np.prod(dims)
    tr_supermap = d**2

    S = cp.Variable((D, D), hermitian=True)
    # {P, S-P} is a tester
    P = cp.Variable((D, D), hermitian=True)
    p_err = cp.Variable()

    def partial_discard(S, trace_list):
        return qcvx.partial_trace(S, dims, trace_list, discard=False)
    
    def controlled_gates(p):
        def normalize(U):
            return U * np.sqrt(U.shape[0] / np.einsum(U, [0,1], U.conjugate(), [0,1]))
        from scipy.linalg import block_diag
        U = block_diag(np.eye(p[0].shape[0]), normalize(p[0]))
        V = block_diag(np.eye(p[1].shape[0]), normalize(p[1]))

        v = np.einsum(U, range(2), V, range(2,4))
        return np.einsum(v, range(4), v.conjugate(), range(4,8)).reshape(4**4,-1)

    cons = [
        P >> 0,
        S >> P,
        cp.trace(S) == tr_supermap,
        S == partial_discard(S, [3]) + partial_discard(S, [1]) - partial_discard(S, [1,3]),
        partial_discard(S, [0,1]) == partial_discard(S, [0,1,3]),
        partial_discard(S, [2,3]) == partial_discard(S, [1,2,3]),
    ]

    # probability of the measurement outcome
    def pr(x):
        return cp.real(cp.trace(P @ controlled_gates(x)))

    # the worst-case error probability
    worst_cons = [pr(x) >= 1-p_err for x in S0]
    worst_cons += [pr(x) <= p_err for x in S1]

    # the avarage error probability
    avg_cons = [
        p_err == sum([1-pr(x) for x in S0] + [pr(x) for x in S1]) / (len(S0) + len(S1))
    ]

    return cp.Problem(cp.Minimize(p_err), cons + worst_cons), cp.Problem(cp.Minimize(p_err), cons + avg_cons)

solve_sdp(*non_causal_strategy2(), 'strategy with indefinite causal order and controlled gates')

strategy with indefinite causal order and controlled gates (1e-06)
    worst-case: 0.05600139526812819
    average: 0.043482475699441074
